In [1]:
import requests
import json
import pandas as pd

The following function gets the boxscore stats of the game which corresponds to the parameter "id."
The goalie stats are calculated locally because sometimes, goalies are pulled in the middle of the game and therefore, we will have to account for the save percentages of two goalies. The number of saves that each goalie made is added and the total is divided by the number of shots taken by the opposing team. All stats except goals only represent regulation time.

In [12]:
def getGameStats(id):
    # get response and convert to json format
    query = 'https://statsapi.web.nhl.com/api/v1/game/' + str(id) + '/boxscore'
    gameStats = requests.get(query)
    gameStats = json.loads(gameStats.text)

    homeName = gameStats['teams']['home']['team']['name']
    awayName = gameStats['teams']['away']['team']['name']
    if homeName == 'Montréal Canadiens': # french accents cause problems in csv format
        homeName = 'Montreal Canadians'
    elif awayName == 'Montréal Canadiens':
        awayName = 'Montreal Canadians'

    awaySkaterStats = gameStats['teams']['away']['teamStats']['teamSkaterStats']
    awayGoals = awaySkaterStats['goals']
    awayPIM = awaySkaterStats['pim']
    awayShots = awaySkaterStats['shots']
    awayPPPercentage = awaySkaterStats['powerPlayPercentage']
    awayPPGoals = awaySkaterStats['powerPlayGoals']
    awayPowerPlays = awaySkaterStats['powerPlayOpportunities']
    awayFaceoffPercentage = awaySkaterStats['faceOffWinPercentage']
    awayBlocked = awaySkaterStats['blocked']
    awayTakeAways = awaySkaterStats['takeaways']
    awayGiveAways = awaySkaterStats['giveaways']
    awayHits = awaySkaterStats['hits']

    # save and shot tallies for calculating overall away goalie stats
    awaySaves = 0
    awayPPSaves = 0
    awayShortHandedSaves = 0
    awayEvenStrengthSaves = 0
    awayShortHandedSA = 0
    awayEvenStrengthSA = 0
    awayPPSA = 0

    awayGameRoster = gameStats['teams']['away']['players']
    for player in awayGameRoster:
        position = awayGameRoster[player]['position']['code']
        if position == 'G':
            
            saves = awayGameRoster[player]['stats']['goalieStats']['saves']
            awaySaves += saves

            ppSaves = awayGameRoster[player]['stats']['goalieStats']['powerPlaySaves']
            awayPPSaves += ppSaves

            shortHandedSaves = awayGameRoster[player]['stats']['goalieStats']['shortHandedSaves']
            awayShortHandedSaves += shortHandedSaves

            evenSaves = awayGameRoster[player]['stats']['goalieStats']['evenSaves']
            awayEvenStrengthSaves += evenSaves

            shortHandedSA = awayGameRoster[player]['stats']['goalieStats']['shortHandedShotsAgainst']
            awayShortHandedSA += shortHandedSA

            evenSA = awayGameRoster[player]['stats']['goalieStats']['evenShotsAgainst']
            awayEvenStrengthSA += evenSA

            ppSA = awayGameRoster[player]['stats']['goalieStats']['powerPlayShotsAgainst']
            awayPPSA += ppSA

    homeSkaterStats = gameStats['teams']['home']['teamStats']['teamSkaterStats']
    homeGoals = homeSkaterStats['goals']
    homePIM = homeSkaterStats['pim']
    homeShots = homeSkaterStats['shots']
    homePPPercentage = homeSkaterStats['powerPlayPercentage']
    homePPGoals = homeSkaterStats['powerPlayGoals']
    homePowerPlays = homeSkaterStats['powerPlayOpportunities']
    homeFaceoffPercentage = homeSkaterStats['faceOffWinPercentage']
    homeBlocked = homeSkaterStats['blocked']
    homeTakeAways = homeSkaterStats['takeaways']
    homeGiveAways = homeSkaterStats['giveaways']
    homeHits = homeSkaterStats['hits']

    # save and shot tallies for calculating overall home goalie stats
    homeSaves = 0
    homePPSaves = 0
    homeShortHandedSaves = 0
    homeEvenStrengthSaves = 0
    homeShortHandedSA = 0
    homeEvenStrengthSA = 0
    homePPSA = 0

    homeGameRoster = (gameStats['teams']['home']['players'])
    for player in homeGameRoster:
        position = homeGameRoster[player]['position']['code']
        if position == 'G':
            saves = homeGameRoster[player]['stats']['goalieStats']['saves']
            homeSaves += saves

            ppSaves = homeGameRoster[player]['stats']['goalieStats']['powerPlaySaves']
            homePPSaves += ppSaves

            shortHandedSaves = homeGameRoster[player]['stats']['goalieStats']['shortHandedSaves']
            homeShortHandedSaves += shortHandedSaves

            evenSaves = homeGameRoster[player]['stats']['goalieStats']['evenSaves']
            homeEvenStrengthSaves += evenSaves

            shortHandedSA = homeGameRoster[player]['stats']['goalieStats']['shortHandedShotsAgainst']
            homeShortHandedSA += shortHandedSA

            evenSA = homeGameRoster[player]['stats']['goalieStats']['evenShotsAgainst']
            homeEvenStrengthSA += evenSA

            ppSA = homeGameRoster[player]['stats']['goalieStats']['powerPlayShotsAgainst']
            homePPSA += ppSA

    awaySavePercentage = awaySaves / (awayEvenStrengthSA + awayShortHandedSA + awayPPSA)
    if awayShortHandedSA != 0:
        awayPPSavePercentage = awayShortHandedSaves / awayShortHandedSA
    else:
        awayPPSavePercentage = 1
    if awayPPSA != 0:
        awayShortHandedSavePercentage = awayPPSaves / awayPPSA
    else:
        awayShortHandedSavePercentage = 1
    if awayEvenStrengthSA != 0:
        awayEvenStrengthSavePercentage = awayEvenStrengthSaves / awayEvenStrengthSA
    else:
        awayEvenStrengthSavePercentage = 1

    homeSavePercentage = homeSaves / (homeEvenStrengthSA + homeShortHandedSA + homePPSA)
    if homeShortHandedSA != 0:
        homePPSavePercentage = homeShortHandedSaves / homeShortHandedSA
    else:
        homePPSavePercentage = 1
    if homePPSA != 0:
        homeShortHandedSavePercentage = homePPSaves / homePPSA
    else:
        homeShortHandedSavePercentage = 1
    if homeEvenStrengthSA != 0:
        homeEvenStrengthSavePercentage = homeEvenStrengthSaves / homeEvenStrengthSA
    else:
        homeEvenStrengthSavePercentage = 1

    gameData = {'id': id, 'homeTeam': homeName, 'awayTeam': awayName, 'homeGoals': homeGoals, 'homePIM': homePIM,
                'homeShots': homeShots, 'homePP%': homePPPercentage, 'homePPG': homePPGoals, 'homePowerPlays': homePowerPlays,
                'homeFaceoff%': homeFaceoffPercentage, 'homeBlockedShots': homeBlocked, 'homeTakeAways': homeTakeAways,
                'homeGiveAways': homeGiveAways, 'homeHits': homeHits, 'homeSaves': homeSaves, 'homePPSaves': homePPSaves,
                'homeShortHandedSaves': homeShortHandedSaves, 'homeEvenStrengthSaves': homeEvenStrengthSaves, 
                'homeShortHandedSA': homeShortHandedSA, 'homeEvenStrengthSA': homeEvenStrengthSA, 'homePPSA': homePPSA, 
                'homeSavePercentage': homeSavePercentage, 'homePPSavePercentage': homePPSavePercentage,
                'homeEvenStrengthSavePercentage': homeEvenStrengthSavePercentage,
                'homeShortHandedSavePercentage': homeShortHandedSavePercentage,
                'awayGoals': awayGoals, 'awayPIM': awayPIM, 'awayShots': awayShots, 'awayPP%': awayPPPercentage,
                'awayPPG': awayPPGoals, 'awayFaceoff%': awayFaceoffPercentage, 'awayBlockedShots': awayBlocked, 
                'awayTakeAways': awayTakeAways, 'awayGiveAways': awayGiveAways, 'awayHits': awayHits,
                'awayPowerPlays': awayPowerPlays, 'awaySaves': awaySaves, 'awayPPSaves': awayPPSaves,
                'awayShortHandedSaves': awayShortHandedSaves, 'awayEvenStrengthSaves': awayEvenStrengthSaves,
                'awayShortHandedSA': awayShortHandedSA, 'awayEvenStrengthSA': awayEvenStrengthSA, 'awayPPSA': awayPPSA,
                'awaySavePercentage': awaySavePercentage, 'awayPPSavePercentage': awayPPSavePercentage,
                'awayEvenStrengthSavePercentage': awayEvenStrengthSavePercentage,
                'awayShortHandedSavePercentage': awayShortHandedSavePercentage, 'OT': False}
    
    if homeGoals == awayGoals:
        query = 'https://statsapi.web.nhl.com/api/v1/game/' + '2015020019' + '/linescore'
        overTimeStats = requests.get(query)
        overTimeStats = json.loads(overTimeStats.text)
        overTimeStats = overTimeStats['teams']
        homeGoals = overTimeStats['home']['goals']
        awayGoals = overTimeStats['away']['goals']
        gameData['homeGoals'] = homeGoals
        gameData['awayGoals'] = awayGoals
        gameData['OT'] = True
    
    return gameData

In [3]:
games20152016 = []
for i in range(2015020001, 2015021231):
    gameData = getGameStats(i)
    games20152016.append(gameData)
gamesDF = pd.DataFrame(games20152016)
gamesDF.to_csv('./gameDataBySeason/NHL20152016.csv', index=False, encoding='utf-8')

In [25]:
games20162017 = []
for i in range(2016020001, 2016021231):
    gameData = getGameStats(i)
    games20162017.append(gameData)
gamesDF = pd.DataFrame(games20162017)
gamesDF.to_csv('./gameDataBySeason/NHL20162017.csv', index=False, encoding='utf-8')    

In [26]:
games20172018 = []
for i in range(2017020001, 2017021272):
    gameData = getGameStats(i)
    games20172018.append(gameData)
gamesDF = pd.DataFrame(games20172018)
gamesDF.to_csv('./gameDataBySeason/NHL20172018.csv', index=False, encoding='utf-8')

In [27]:
games20182019 = []
for i in range(2018020001, 2018021272):
    gameData = getGameStats(i)
    games20182019.append(gameData)
gamesDF = pd.DataFrame(games20182019)
gamesDF.to_csv('./gameDataBySeason/NHL20182019.csv', index=False, encoding='utf-8')

In [28]:
games20192020 = []
for i in range(2019020001, 2019021083):
    gameData = getGameStats(i)
    games20192020.append(gameData)
gamesDF = pd.DataFrame(games20192020)
gamesDF.to_csv('./gameDataBySeason/NHL20192020.csv', index=False, encoding='utf-8')

In [29]:
games20202021 = []
for i in range(2020020001, 2020020869):
    gameData = getGameStats(i)
    games20202021.append(gameData)
gamesDF = pd.DataFrame(games20202021)
gamesDF.to_csv('./gameDataBySeason/NHL20202021.csv', index=False, encoding='utf-8')

In [14]:
games20212022 = []
for i in range(2021020001, 2021021312):
    gameData = getGameStats(i)
    games20212022.append(gameData)
gamesDF = pd.DataFrame(games20212022)
gamesDF.to_csv('./gameDataBySeason/NHL20212022.csv', index=False, encoding='utf-8')